In [ ]:
#Pacotes utilizados
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix
from keras import models, optimizers, layers, losses, utils
from sklearn.ensemble import RandomForestClassifier

'''Função que recebe o endereço do pickle e retorna o conteúdo no formado de numpy array'''
def read_pickle(name):
    with (open(name, 'rb')) as openfile:
        while True:
            try:
                one_instance = pickle.load(openfile)
            except EOFError:
                break
    one_instance = np.asanyarray(one_instance)
    return one_instance

'''Função que recebe os espectogramas e os labels e transforma para o formato de entrada da rede aceito pelo Keras'''
def pre_processing_datas_to_cnn_format(X, y):
    X = X.reshape(X.shape[0],X.shape[1],X.shape[2],1)
    y = utils.to_categorical(y)
    return X, y

'''Função que cria a arquitetura de rede'''
def create_model(shape_in, num_classes = 10, dropout_value = 0.5):
    model = models.Sequential()

    #Primeira camada convolucional
    model.add(layers.Conv2D(32, kernel_size=(5,5), activation = 'relu', padding='same', input_shape=(shape_in.shape[0],shape_in.shape[1],shape_in.shape[2]), name = 'conv_1'))

    #Primeira camada de pooling
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name = 'pool_1', padding='same'))
    
    model.add(layers.BatchNormalization())
    
    #Segunda camada convolucional
    model.add(layers.Conv2D(64, kernel_size=(5,5), activation = 'relu', padding='same', name = 'conv_2'))

    #Segunda camada de pooling
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name = 'pool_2', padding='same'))
    
    model.add(layers.BatchNormalization())
    
    #Terceira camada convolucional
    model.add(layers.Conv2D(64, kernel_size=(5,5), activation = 'relu', padding='same', name = 'conv_3'))

    #Terceira camada de pooling
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name = 'pool_3', padding='same'))
    
    model.add(layers.BatchNormalization())

    model.add(layers.Flatten(name = 'flatten'))
    
    #Dropout
    model.add(layers.Dropout(dropout_value))

    #Camada totalmente conectada
    model.add(layers.Dense(512, name = 'dense_1', activation='relu'))
    
    #Dropout
    model.add(layers.Dropout(dropout_value))

    #Camada de saída
    model.add(layers.Dense(num_classes, activation='softmax', name = 'classification'))
    
    #Para visualizar a arquitetura da rede
    model.summary()
    return model

'''Função que recebe os espectogramas e o endereço do modelo treinado e retorna os características obtidas, que são as saídas da penultima camada da rede'''
def extract_features(X_test, path_model):
    model = models.load_model(path_model)

    intermediate_layer_model = models.Model(inputs=model.input, outputs=model.get_layer(index = -2).output)
    features = intermediate_layer_model.predict(X_test)

    features = pd.DataFrame(data=features)
    
    return features

In [ ]:
#Endereços dos pickles contendo os espectogramas e labels
path_train = '/data/deborah/UrbanSound8K/log_specs_100_treino.pickle'
path_train_labels = '/data/deborah/UrbanSound8K/labels_100_treino.pickle'
path_test = '/data/deborah/UrbanSound8K/log_specs_100_teste.pickle'
path_test_labels = '/data/deborah/UrbanSound8K/labels_100_teste.pickle'

#Nome do modelo que será salvo
path_model = 'trained_model_100_augmented.h5'
learning_rate = 0.0001
epochs = 1
batch_size = 128
num_classes = 10
dropout_value = 0.5
decay_=1e-3

X_train = read_pickle(path_train)
y_train = read_pickle(path_train_labels)
X_test = read_pickle(path_test)
y_test = read_pickle(path_test_labels)

#Transformando os espectogramas e labels
X_train, y_train = pre_processing_datas_to_cnn_format(X_train, y_train)
X_test, y_test = pre_processing_datas_to_cnn_format(X_test, y_test)

In [ ]:
#Criando o modelo e configurando os parâmetros de treinamento
model = models.load_model(path_model)
#model = create_model(X_train[0], num_classes, dropout_value)
sgd = optimizers.SGD(lr=learning_rate, decay=decay_)
model.compile(loss=losses.categorical_crossentropy, optimizer=sgd, metrics=['accuracy'])

#Treinamento
model.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size=batch_size, epochs=epochs, verbose=1)

#Salvando o modelo treinado
#model.save(path_model) 


In [ ]:
#Lendo o modelo salvo após o treinamento
model = models.load_model(path_model)

#Fazendo a predição dos dados do conjunto de teste com a rede treinada
pred = model.predict_classes(X_test)

#Imprimindo a acuracia e a matriz de confusão
print('Acuracia da rede = ',accuracy_score(pred,np.argmax(y_test,axis=1)))
print('Matriz de confusão da rede = \n', confusion_matrix(pred,np.argmax(y_test,axis=1)))

In [ ]:
#Utilizando a rede treinada para extrair as características dos espectogramas
X_train_features = extract_features(X_train,path_model)
X_test_features = extract_features(X_test,path_model)

#Treinando uma Random Forest
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train_features,np.argmax(y_train,axis=1))

#Predição com a Random Forest treinada
predicted = clf.predict(X_test_features)

#Imprimindo a acuracia e a matriz de confusão
print('Acuracia da Random Forest = ',accuracy_score(predicted,np.argmax(y_test,axis=1)))
print('Matriz de confusão da Random Forest = \n',confusion_matrix(predicted,np.argmax(y_test,axis=1)))

In [ ]:
a
epochs = 100
model.fit(X_train, y_train, validation_data = (X_test, y_test),
          batch_size=batch_size, epochs=epochs, verbose=1)

model.save(path_model) 

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

model = models.load_model(path_model)

pred = model.predict_classes(X_test)
print('Acuracia = ',accuracy_score(pred,np.argmax(y_test,axis=1)))
print(confusion_matrix(pred,np.argmax(y_test,axis=1)))

In [ ]:
'''Função que recebe os espectogramas e o endereço do modelo treinado 
e retorna os características obtidas, que são as saídas da penultima
camada da rede'''
def extract_features(X_test, path_model):
    model = load_model(path_model)

    intermediate_layer_model = Model(inputs=model.input,
                                     outputs=model.get_layer(index = -2).output)
    features = intermediate_layer_model.predict(X_test)

    features = pd.DataFrame(data=features)
    
    return features

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train = read_pickle(path_train)
y_train = read_pickle(path_train_labels)
X_test = read_pickle(path_test)
y_test = read_pickle(path_test_labels)

X_train, _ = pre_processing_datas_to_cnn_format(X_train, y_train)
X_test, _ = pre_processing_datas_to_cnn_format(X_test, y_test)

X_train_features = extract_features(path_model,X_train,'')
X_test_features = extract_features(path_model,X_test,'')

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train_features,y_train)

In [ ]:
predicted = clf.predict(X_test_features)

print('Acuracia = ',accuracy_score(predicted,y_test))
confusion_matrix(predicted,y_test)